In [2]:
import requests

In [ ]:
class USDA_API():
    def __init__(self, key):
        self.url = 'https://quickstats.nass.usda.gov/api'
        self.key = key
        self.params = ''
        self.format = 'json'

    
    def add_params(self, fieldname, value):
        self.params += f'&{fieldname}={value}'
    
    def return_params(self):
        return self.params
    
    def return_call(self):
        return self.url + '/api_GET/?' + f'key={self.key}' + self.params + f'&format={self.format}'

    def remove_params(self, fieldname):
        if len(self.params.split('&')) > 1:
            new_params = ''
            size = 1
            remove_params = [item for item in self.params.split('&') if fieldname not in item]
        
            for item in remove_params:
                if len(remove_params) > 1 and len(item) != 0 and size < len(remove_params - 1):
                    new_params += item + '&'
                    size = size + 1
                else:
                    new_params  += item
            self.params = '&' + new_params
        else:
            self.params = self.params
            print('No Parameters to remove')

    def call(self):
        try:
            response = requests.get(f'{self.url}/api_GET/?key={self.key}{self.params}&format={self.format}')
            response.raise_for_status()
            if response.status_code == 200:
                get_counts = requests.get(f'{self.url}/get_counts/?key={self.key}{self.params}').json()
            
                if get_counts['count'] >= 50000:
                    return f'Unable to Process Request. Request is greater than 50000 rows'
                else:
                    return response.json()
                
        except requests.exceptions.HTTPError as errh:
            if   response.status_code == 415:
                return f'Unable to Process Request. [415]: ' + str(response.json()['error'][0])
            elif response.status_code == 401:
                return f'Unable to Process Request. [401]: ' + str(response.json()['error'][0])


In [236]:
data = USDA_API('9DCFA99E-5354-384A-8E24-52D49A261AB0')

data.add_params('commodity_desc','CORN')
data.add_params('state_alpha','VA')
data.add_params('year','2018')

data.call()

{'data': [{'congr_district_code': '',
   'state_name': 'VIRGINIA',
   'week_ending': '',
   'prodn_practice_desc': 'IRRIGATED',
   'county_name': '',
   'statisticcat_desc': 'AREA HARVESTED',
   'county_ansi': '',
   'Value': '11,466',
   'CV (%)': '23.7',
   'country_name': 'UNITED STATES',
   'begin_code': '00',
   'end_code': '00',
   'state_alpha': 'VA',
   'year': 2018,
   'domaincat_desc': 'IRRIGATION METHOD, PRIMARY: (PRESSURE)',
   'watershed_desc': '',
   'watershed_code': '00000000',
   'county_code': '',
   'country_code': '9000',
   'sector_desc': 'CROPS',
   'region_desc': '',
   'asd_code': '',
   'util_practice_desc': 'GRAIN',
   'location_desc': 'VIRGINIA',
   'asd_desc': '',
   'freq_desc': 'ANNUAL',
   'state_fips_code': '51',
   'domain_desc': 'IRRIGATION METHOD, PRIMARY',
   'group_desc': 'FIELD CROPS',
   'source_desc': 'CENSUS',
   'class_desc': 'ALL CLASSES',
   'state_ansi': '51',
   'short_desc': 'CORN, GRAIN, IRRIGATED - ACRES HARVESTED',
   'unit_desc': 'ACRE

In [ ]:
error_message = {
    
}